In [ ]:
import random

import yaml

In [ ]:
class TheoreticExercise:
    def __init__(self, exercise_collection_path):
        with open(exercise_collection_path) as f:
            self._exercises = yaml.load(f)['exercises']
        self.generate()

    def generate(self):
        self._exercise = random.choice(self._exercises)

    @property
    def question(self):
        return self._exercise['question']

    @property
    def answer(self):
        return self._exercise['answer']

In [ ]:
te = TheoreticExercise('exercises/szamabrazolas.yaml')
print(te.question)
print(te.answer)

In [ ]:
class FloatExercise:
    """
    IEEE Float exercise
    """
    
    def __init__(self):
        self.generate()
        self.calculate()

    def generate(self):
        self._sign = random.choice([0, 1])
        self._frac = random.randint(12, 62)
        self._integral = random.randint(3000, 10000)
        self._is_float_to_hex = random.choice([False, True])

    def calculate(self):
        integral_bin = f'{self._integral:b}'
        self._exponent = len(integral_bin) - 1
        shft = self._exponent + 127
        integral_bin = integral_bin[1:]
        fine = f'{self._sign}{shft:08b}{integral_bin}{self._frac:06b}'
        fine = fine + '0' * (32 - len(fine))
        self._value = self._integral + self._frac / 64
        if self._sign == 1:
            self._value *= -1
        self._hex = f'{int(fine, 2):X}'
    
    @property
    def question(self):
        if self._is_float_to_hex:
            return self.float_question()
        else:
            return self.hex_question()
    
    @property
    def answer(self):
        if self._is_float_to_hex:
            return self.hex_answer()
        else:
            return self.float_answer()

    def float_question(self):
        return f"""
        Milyen byte-okon ábrázolható a {self._value} érték egyszeres lebegőpontos számábrázolással?
        """

    def hex_question(self):
        return f"""
        Milyen értéket ábrázolnak a \\texttt{ {self._hex} } byte-ok
        egyszeres lebegőpontos számábrázolást feltételezve?
        """

    def float_answer(self):
        return f"""
        Az ábrázolt érték a {self._value}. (A kitevő {self._exponent}.)
        """

    def hex_answer(self):
        return f"""
        Az \\texttt{ {self._hex} } byte-okon ábrázolható. (A kitevő {self._exponent}.)
        """

In [ ]:
fe = FloatExercise()
print(fe.question)
print(fe.answer)

In [ ]:
class EuclideanExercise:
    """
    Extended Euclidean algorithm exercise
    """
    
    def __init__(self):
        self.generate()
        self.calculate()
    
    def generate(self):
        n = 0
        while n < 4 or n > 7:
            a_orig = random.randint(1000, 2000)
            b_orig = random.randint(200, 1000)
            a = a_orig
            b = b_orig
            n = 0
            while b:
                a, b = b, a % b
                n += 1
        self._a = a_orig
        self._b = b_orig

    def calculate(self):
        self._d, self._x, self._y = self.calc_representation(self._a, self._b)
    
    def calc_representation(self, a, b):
        if b > 0:
            d, x, y = self.calc_representation(b, a % b)
            x, y = y, x - y * (a // b)
        else:
            d = a
            x = 1
            y = 0
        return d, x, y

    def __str__(self):
        return f"({self._a}, {self._b}) -> ({self._d}, {self._x}, {self._y})" 
    
    @property
    def question(self):
        return f"""
        Számítsa ki az {self._a} és {self._b} értékek
        legnagyobb közös osztóját, majd írja fel azt az értékek egész együtthatós lineáris kombinációjaként!
        """
    
    @property
    def answer(self):
        return f"""
        A legnagyobb közös osztó {self._d}.
        Az együtthatók $x^{{*}} = {self._x}, y^{{*}} = {self._y}$.
        A reprezentációs tétel alapján
        ${self._d} = {self._a} \\cdot {self._x} + {self._b} \\cdot {self._y}$
        """

In [ ]:
ee = EuclideanExercise()
print(ee.question)
print(ee.answer)

In [ ]:
def linear_hash(k, i, m):
    index = (k + i) % m
    return index

def quadratic_hash(k, i, m):
    index = ((k % m) + ((i * (i + 1)) // 2)) % m
    return index

def double_hash(k, i, m):
    h_0 = k % m
    h_1 = 1 + (k % (m - 1))
    index = (h_0 + i * h_1) % m
    return index

class HashTableExercise:
    """
    Hash table exercise
    """
    
    def __init__(self):
        self.generate()

    def generate(self):
        self._max_trials = 0
        while self._max_trials not in [2, 3, 4]:
            self._h_name = random.choice(['linear', 'quadratic', 'double'])
            self._h = {
                'linear': linear_hash,
                'quadratic': quadratic_hash,
                'double': double_hash
            }[self._h_name]
            self._m = random.choice([7, 11])
            self._table = [{'key': [], 'state': ['sz']} for _ in range(self._m)]
            self._operations = self.generate_operations()
            self.process_operations()

    def generate_operations(self):
        n_firsts = 7 if self._m == 7 else 9
        insertables = [1, 1]
        while len(insertables) != len(set(insertables)):
            insertables = [random.randint(50, 999) for _ in range(n_firsts)]
        removables = random.sample(insertables[:5], 2)
        items = [str(k) for k in insertables]
        items.insert(5, 'T{}'.format(removables[0]))
        items.insert(7, 'T{}'.format(removables[1]))
        return ', '.join(items)

    def process_operations(self):
        max_trials = 0
        for operation in self._operations.split(', '):
            if operation[0] != 'T':
                key = int(operation)
                n_trials = self.insert_key(key)
            else:
                key = int(operation[1:])
                n_trials = self.remove_key(key)
            if n_trials > max_trials:
                max_trials = n_trials
        self._max_trials = max_trials

    def insert_key(self, k):
        is_inserted = False
        n_trial = 0
        while not is_inserted:
            row = self._h(k, n_trial, self._m)
            if self._table[row]['state'][-1] in ['sz', 't']:
                self._table[row]['key'].append(k)
                self._table[row]['state'].append('f')
                is_inserted = True
            n_trial += 1
            if n_trial > self._m:
                return n_trial
        return n_trial - 1

    def remove_key(self, k):
        is_removed = False
        n_trial = 0
        while not is_removed:
            row = self._h(k, n_trial, self._m)
            if self._table[row]['state'][-1] == 'sz':
                is_removed = True
            elif self._table[row]['state'][-1] == 't':
                pass
            elif self._table[row]['state'][-1] == 'f':
                if self._table[row]['key'][-1] == k:
                    self._table[row]['state'].append('t')
                    is_removed = True
            n_trial += 1
            if n_trial > self._m:
                return n_trial
        return n_trial - 1

    @property
    def question(self):
        mode = {
            'linear': 'lineáris kipróbálási sorozatot',
            'quadratic': 'négyzetes kipróbálási sorozatot',
            'double': 'dupla hasítást'
        }[self._h_name]
        return f"""
        Szúrja be a következő kulcsokat egy {self._m} méretű, nyílt címzésű hasítótáblába {mode} alkalmazva!
        (A T betűk a kulcsok előtt törlést jelentenek.)
        $$
        {self._operations}
        $$
        Mennyi volt a kipróbálási sorozat maximális $i$ indexe?
        """
    
    @property
    def answer(self):
        table_tex = "\\begin{tabular}{|l|l|l|}\n"
        table_tex += "index & foglaltság & kulcs \\\\\n"
        table_tex += "\\hline\n"
        for index, row in enumerate(self._table):
            table_tex += f"{index} & " + ', '.join([s for s in row['state']]) + " & " + ', '.join([str(k) for k in row['key']]) + " \\\\\n"
        table_tex += "\\hline\n"
        table_tex += "\\end{tabular}"
        return f"""{table_tex}
        \\\\
        A kipróbálási sorozat maximális $i$ indexe {self._max_trials}.
        """

In [ ]:
hte = HashTableExercise()
print(hte.question)
print(hte.answer)

In [ ]:
class QuickSortExercise:
    """
    Quick sort exercise
    """
    
    def __init__(self):
        self.generate()
        self.calculate()

    def generate(self):
        n_items = random.choice([5, 6])
        self._values = random.sample([i for i in range(10, 90)], n_items)

    def calculate(self):
        self._n_qs_calls = 0
        self._n_divide_calls = 0
        self._n_swaps = 0
        self.quick_sort(0, len(self._values) - 1)

    def quick_sort(self, p, r):
        self._n_qs_calls += 1
        if p < r:
            q = self.divide(p, r, self._values[p])
            self.quick_sort(p, q)
            self.quick_sort(q + 1, r)

    def divide(self, p, r, x):
        self._n_divide_calls += 1
        i = p - 1
        j = r + 1
        while True:
            i += 1
            while self._values[i] < x:
                i += 1
            j -= 1
            while self._values[j] > x:
                j -= 1
            if i < j:
                t = self._values[i]
                self._values[i] = self._values[j]
                self._values[j] = t
                self._n_swaps += 1
            else:
                return j

    @property
    def question(self):
        return f"""
        Gyorsrendezéssel rendezze az $A = {self._values}$ tömb elemeit! \\\\
        Mennyi csere és mennyi rekurzív hívás történt a rendezés során? \\\\
        Mennyi \\texttt{{FELOSZT}} eljárás hívásra volt szükség?
        """
    
    @property
    def answer(self):
        return f"""
        \\begin{{itemize}}
            \\item Cserék száma: {self._n_swaps}
            \\item Rekurzív hívások száma: {self._n_qs_calls - 1}
            \\item \\texttt{{FELOSZT}} eljáráshívások száma: {self._n_divide_calls}
        \\end{{itemize}}
        """

In [ ]:
qse = QuickSortExercise()
print(qse.question)
print(qse.answer)

In [ ]:
class BinSortExercise:
    """
    Bin sort exercise
    """
    
    def __init__(self):
        self.generate()
        self.calculate()

    def generate(self):
        n_items = random.randint(7, 9)
        n_uniques = n_items - 3
        self._values = [i for i in range(1, n_uniques + 1)]
        for _ in range(3):
            self._values.append(random.randrange(n_uniques) + 1)
        random.shuffle(self._values)
        self._values = [3, 1, 6, 4, 3, 4, 4, 2]

    def calculate(self):
        c = [0] * max(self._values)
        self._c_1 = c[:]
        for i in self._values:
            c[i - 1] += 1
        self._c_2 = c[:]
        for i in range(1, len(c)):
            c[i] = c[i] + c[i - 1]
        self._c_3 = c[:]
        for j in range(len(self._values) - 1, -1, -1):
            c[self._values[j] - 1] -= 1
        self._c_4 = c[:]

    @property
    def question(self):
        return f"""
        Leszámláló rendezéssel rendezze az $A = {self._values}$ tömb elemeit! \\\\
        Írja fel, hogy az egyes lépések után hogyan milyen értékeket tartalmazott a $C$ segédtömb!
        """

    @property
    def answer(self):
        return f"""
        \\begin{{itemize}}
            \\item 1. lépés után: $C = {self._c_1}$
            \\item 2. lépés után: $C = {self._c_2}$
            \\item 3. lépés után: $C = {self._c_3}$
            \\item 4. lépés után: $C = {self._c_4}$
        \\end{{itemize}}
        """

In [ ]:
bse = BinSortExercise()
print(bse.question)
print(bse.answer)

In [ ]:
huffman_words = [
    'MENTHETETLENNEK',
    'IDE ODA AMODA',
    'MUTATHATTA',
    'MUTATHATATLAN',
    'HALMAZALAKKAL',
    'HELLO BELLO',
    'HINTATALAN',
    'TAVASSZAL ASZAL',
    'REMETELELKEK',
    'ETTE VETTE TEREMTETTE',
    'ELTEREGETETTE',
    'IDEDIDERGETT',
    'ELRETTENTHETETLEN',
    'MUNKARUHA',
    'BIRREG BERREG',
    'TIC TAC TOE',
    'REBESGETETT',
    'HIDEGELEDELLEL',
    'BARRAKUDA BARAKK',
    'ELLEHETETLENEDTEM',
    'AABACAXDDACD',
    'NEM LEHET TELHETETLEN',
    'ALAPTALANNAL',
    'MEDERBE DERMEDVE',
    'REMEMBER ME',
    'INTERNETEN',
    'MEREDEKEBBEN',
    'KALAPPAPLAN',
    'RUMMALRUMBA',
    'SZEREP CSERE',
    'SZELEPCSERE',
    'XYZZYZYOZZXXT',
    'BAKABAKKANCS',
    'ENTER ENTER',
    'OLVASZTOTT',
    'TINTA MINTA',
    'HEVEDERVEDER',
    'SZENTSZIGETEN',
    'ELREPEDETLEN',
    'TROCKOS ORKOK',
    'NINE EIN NONE',
    'IFA KALIFA',
    'ERRE ARRA EMERRE',
    'MINDENKI IDE KENI',
    'HAMARABB ARRA',
    'ELHAVAZVA EZZEL AZZAL',
    'NINCS ENNEK SINCS',
    'TRAC ARCCAL',
    'HIDEG RIDEG IDEG',
    'TERMELHETETLENNEL'
]

In [ ]:
class HuffmanExercise:
    """
    Huffman exercise
    """
    
    def __init__(self):
        self.generate()
        self.calculate()

    def generate(self):
        self._word = random.choice(huffman_words)
    
    def calculate(self):
        tree = self.calc_tree(self._word)
        self._codes = {}
        self.calc_codes('', tree)
        self._coded_message = ''
        for symbol in self._word:
            self._coded_message += self._codes[symbol]

    def calc_tree(self, word):
        symbols = set(word)
        tree = [{'symbol': symbol, 'frequency': word.count(symbol)} for symbol in symbols]
        tree.sort(key=lambda x: x['frequency'])
        while len(tree) > 1:
            a = tree.pop(0)
            b = tree.pop(0)
            c = {
                'frequency': a['frequency'] + b['frequency'],
                'children': [a, b]
            }
            tree.append(c)
            tree.sort(key=lambda x: x['frequency'])
        tree = tree[0]
        return tree

    def calc_codes(self, path, node):
        if 'children' in node:
            self.calc_codes(path + '0', node['children'][0])
            self.calc_codes(path + '1', node['children'][1])
        else:
            symbol = node['symbol']
            self._codes[symbol] = path

    @property
    def question(self):
        return f"""Huffman kódolás segítségével kódolja a \\texttt{{ "{self._word}" }} üzenetet! \\\\
        Rajzolja fel a kódfát, írja fel a kódolt üzenetet, adja meg annak hosszát és
        számítsa ki az átlagos kódhosszt!
        """

    @property
    def answer(self):
        table_tex = "\\begin{tabular}{|l|l|l|}\n"
        table_tex += "szimbólum & gyakoriság & kódszó \\\\\n"
        table_tex += "\\hline\n"
        for symbol in set(self._word):
            table_tex += f"{symbol} & {self._word.count(symbol)} & {self._codes[symbol]} \\\\\n"
        table_tex += "\\hline\n"
        table_tex += "\\end{tabular}"
        return f"""{table_tex} \\\\
        Kódolt üzenet: {self._coded_message} \\\\
        Kódolt üzenet hossza: {len(self._coded_message)} \\\\
        Átlagos kódhossz: {len(self._coded_message) / len(self._word)}
        """


In [ ]:
hfe = HuffmanExercise()
print(hfe.question)
print(hfe.answer)

In [ ]:
latex_head = r"""
\documentclass[a4paper]{article}

% Set margins
\usepackage[hmargin=2.5cm, vmargin=3cm]{geometry}

\frenchspacing

% Language packages
\usepackage[utf8]{inputenc}
\usepackage[T1]{fontenc}
\usepackage[magyar]{babel}

% AMS
\usepackage{amssymb,amsmath}

\usepackage{xcolor}

\begin{document}

\pagestyle{empty}
"""

latex_body = r"""
Miskolci Egyetem, Matematikai Intézet

\hskip 10cm Név:

\medskip

\hskip 10cm Neptun-kód:

\begin{center}
   \large \textbf{Zárthelyi dolgozat - \texttt{{{code}}} \\
   ADATSTRUKTÚRÁK ÉS ALGORITMUSOK (GEMAK121-B) c. tárgyból}
\end{center}

\bigskip

\noindent \textbf{1. Feladat} {{exercise1}}
\textit{(1 pont)}

\bigskip

\noindent \textbf{2. Feladat} {{exercise2}}
\textit{(1 pont)}

\bigskip

\noindent \textbf{3. Feladat} {{exercise3}}
\textit{(1 pont)}

\bigskip

\noindent \textbf{4. Feladat} {{exercise4}}
\textit{(1 pont)}

\bigskip

\noindent \textbf{5. Feladat} {{exercise5}}
\textit{(2 pont)}

\bigskip

\noindent \textbf{6. Feladat} {{exercise6}}
\textit{(2 pont)}

\bigskip

\noindent \textbf{7. Feladat} {{exercise7}}
\textit{(2 pont)}

\bigskip

\noindent \textbf{8. Feladat} {{exercise8}}
\textit{(2 pont)}

\bigskip

\noindent Ponthatárok: 0-5 elégtelen, 6 elégséges, 7-8 közepes, 9-10 jó, 11-12 jeles

\newpage
"""

latex_tail = r"""
\end{document}
"""

In [ ]:
def collect_exercises():
    tes = [
        TheoreticExercise('exercises/szamabrazolas.yaml'),
        TheoreticExercise('exercises/adatstrukturak.yaml'),
        TheoreticExercise('exercises/rendezesek.yaml'),
        TheoreticExercise('exercises/pseudo.yaml')
    ]
    gens = [
        FloatExercise(),
        EuclideanExercise(),
        HashTableExercise(),
        QuickSortExercise(),
        BinSortExercise(),
        HuffmanExercise()
    ]
    random.shuffle(gens)
    gens = gens[:4]
    generators = tes + gens
    exercises = [g.question for g in generators]
    solutions = [g.answer for g in generators]
    return exercises, solutions

In [ ]:
with open('/tmp/zh_exercises.tex', 'w') as tex_file:
    with open('/tmp/zh_solutions.tex', 'w') as solution_file:
        tex_file.write(latex_head)
        solution_file.write(latex_head)
        for page_number in range(500):
            code = f'210424{page_number:04d}'
            body = latex_body
            solution_body = latex_body
            body = body.replace('{{code}}', code)
            solution_body = solution_body.replace('{{code}}', code)
            exercises, solutions = collect_exercises()
            for i in range(8):
                body = body.replace('{{exercise' + str(i + 1) + '}}', exercises[i])
                solution_body = solution_body.replace('{{exercise' + str(i + 1) + '}}', exercises[i] + '\n\n\n\n{\\color{blue} ' + str(solutions[i]) + '}\n')
            tex_file.write(body)
            solution_file.write(solution_body)
        tex_file.write(latex_tail)
        solution_file.write(latex_tail)

In [ ]:
!pdflatex /tmp/zh_exercises.tex
!pdflatex /tmp/zh_solutions.tex